In [1]:
import requests

def get_json_data(url):
    try:
        # Make a GET request to the specified URL
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON data from the response
            json_data = response.json()
            return json_data
        else:
            print(f"Error: Unable to fetch data. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def list_variables(json_data, parent_key=''):
    # Recursively iterate through the JSON data and print variable names
    for key, value in json_data.items():
        full_key = f"{parent_key}.{key}" if parent_key else key
        if isinstance(value, dict):
            list_variables(value, parent_key=full_key)
        else:
            print(full_key)

if __name__ == "__main__":
    # Replace the URL with the actual URL of the JSON file
    json_url = "https://example.com/path/to/your/json/file.json"

    # Fetch JSON data from the URL
    json_data = get_json_data(json_url)

    # If data is fetched successfully, list variables
    if json_data:
        list_variables(json_data)


Error: Unable to fetch data. Status code: 404


In [2]:
import json

# Specify the path to your JSON file
json_file_path = 'C:/Users/adop/OneDrive - Danmarks Tekniske Universitet/Dokumenter/GITHUB/SDN/credentials.json'

try:
    # Open the JSON file for reading
    with open(json_file_path, 'r') as file:
        # Load the JSON data
        json_data = json.load(file)
        
        # Now you can work with the JSON data as a Python dictionary
        # print(json_data)
        
        # Access specific values using keys
        # For example, if the JSON has a key 'example_key', you can access its value like this:
        # example_value = json_data['example_key']
        # print(example_value)
except FileNotFoundError:
    print(f"Error: File not found at {json_file_path}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

list_variables(json_data)

user1
user2
user3


In [5]:
def list_variables(json_data, parent_key=''):
    # Recursively iterate through the JSON data and print variable names
    for key, value in json_data.items():
        full_key = f"{parent_key}.{key}" if parent_key else key
        if isinstance(value, dict):
            list_variables(value, parent_key=full_key)
        else:
            print(full_key)